In [1]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
#import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statsmodels.api as sm
%matplotlib inline

###This notebook will focus on the analytical part of the 311 demographics analysis: that is, multilinear regressions between resident and working population socio-demographic attributes and  the number of 311 calls by type per capita, at the NTA level will be performed and analyzed

In [2]:
#Upload the working population attributes, and the resident population attributes at the CT level.
#We keep both datasets separated in order to generate separate regressions per type of features
demographics_NTA_NYC_residents=pd.read_csv('demographics_nta_NYC_residents_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
demographics_NTA_NYC_workers=pd.read_csv('demographics_nta_NYC_workers_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
print len(demographics_NTA_NYC_residents),len(demographics_NTA_NYC_workers)

195 194


In [3]:
#Upload the 311 calls by type, normalized by total resident population
calls_bytype_normalized=pd.read_csv('Call by type with normalization by resident - NTA level.csv').drop('Unnamed: 0', axis=1)

In [5]:
#Delete the total population from the dataset since we are not going to use it
del(calls_bytype_normalized['Total Population'])

In [6]:
#Upload the 311 calls by type, non-normalized and the total worker poulation

calls_bytype_nonnormalized=pd.read_csv('Call by type without normalization - NTA level.csv').drop('Unnamed: 0', axis=1)
workers = pd.read_csv('demographics_NTA_NYC_workers.csv')

In [8]:
workers = workers[['Neighborhood', 'total workers']]

In [9]:
calls_bytype_nonnormalized = pd.merge(calls_bytype_nonnormalized, workers, on='Neighborhood')

In [10]:
calls_bytype_normalized_workers = calls_bytype_nonnormalized[calls_bytype_nonnormalized.columns[1:]].astype(float).div(calls_bytype_nonnormalized['total workers'].astype(float), axis=0)

In [11]:
calls_bytype_normalized_workers['Neighborhood'] = calls_bytype_nonnormalized['Neighborhood']

In [13]:
#Delete the total population from the dataset since we are not going to use it
del(calls_bytype_normalized_workers['total workers'])

In [16]:
#callsbytype_attributes will be a dataframe combining all the information (demographics + calls by type)
#we created to separate frames, one for residents and one for workers
callsbytype_attributes_residents=pd.merge(calls_bytype_normalized,demographics_NTA_NYC_residents, on='Neighborhood',how='inner')
callsbytype_attributes_workers=pd.merge(calls_bytype_normalized_workers,demographics_NTA_NYC_workers, on='Neighborhood',how='inner')
print len(callsbytype_attributes_residents), len(callsbytype_attributes_workers)

190 190


In [35]:
types_of_calls=calls_bytype_normalized.columns[:-1] #types of calls
types_of_calls_workers=calls_bytype_normalized_workers.columns[:-1] #types of calls workers

In [34]:
types_of_calls_workers

,AGENCY,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,...,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,total_calls,Neighborhood
0,NaN,0.001778,NaN,0.001368,0.007798,NaN,NaN,0.000274,0.000137,NaN,...,0.000274,NaN,0.008208,0.004241,0.000547,0.030233,NaN,NaN,0.726265,Allerton-Pelham Gardens
1,NaN,0.000687,NaN,0.001374,0.005154,0.000344,0.000172,0.000515,0.000172,0.000172,...,0.000172,0.000344,0.000859,0.004982,0.002577,0.046727,NaN,NaN,0.884900,Annadale-Huguenot-Prince's Bay-Eltingville
2,NaN,0.001754,NaN,0.001754,0.012865,NaN,0.000585,0.001170,NaN,0.001170,...,NaN,NaN,0.002339,0.028655,0.006433,0.218129,NaN,NaN,2.339766,Arden Heights
3,NaN,0.002079,0.000041,0.001834,0.004280,NaN,0.000041,0.000571,0.000163,0.000041,...,0.001427,0.000041,0.005381,0.001264,0.000897,0.013779,0.000041,NaN,0.658500,Astoria
4,NaN,0.001335,NaN,0.002224,0.002891,0.000222,NaN,0.000222,0.000445,NaN,...,0.000667,NaN,0.000445,0.004893,0.001112,0.072954,NaN,NaN,0.861655,Auburndale
5,NaN,0.006588,NaN,0.000941,0.017176,NaN,0.000471,0.001176,NaN,NaN,...,0.000471,NaN,0.014588,0.013882,0.001412,0.069647,NaN,NaN,1.671059,Baisley Park
6,NaN,0.004208,NaN,0.000956,0.004782,NaN,0.000574,NaN,NaN,NaN,...,0.000574,0.000191,0.015111,0.001530,0.000765,0.041890,0.000191,NaN,1.113045,Bath Beach
7,NaN,0.000028,NaN,0.000286,0.000050,NaN,0.000009,0.000050,NaN,0.000003,...,0.000233,0.000022,0.000037,0.000019,0.000025,0.000630,0.000006,NaN,0.028178,Battery Park City-Lower Manhattan
8,NaN,0.002320,NaN,0.003574,0.002776,NaN,0.000380,0.000646,0.000038,NaN,...,0.000228,0.000304,0.008784,0.000875,0.000532,0.013689,NaN,NaN,0.679938,Bay Ridge
9,NaN,0.000688,NaN,0.000963,0.002546,NaN,0.000069,0.000482,NaN,0.000069,...,0.000138,NaN,0.002133,0.002890,0.000619,0.027520,NaN,NaN,0.553629,Bayside-Bayside Hills


In [8]:
#Defining the selected features for the innitial resident lasso regression
#we will use this innitial regression to determine a selected list of types of complaints 
#that meet a certain threadshold to then perform a selective regression by feature types
resident_features = [u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over', u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race', u'family households', u'nonfamily households', u'population education high school', 
       u'population education bachelors', u'population education masters', u'population education phd', 'owner  occupied units',  
       u'renter occupied units', u'transportation car', u'transportation public', 'tranportation motorcycle', 
       u'Transportation Other means', u'number of cars', u'household income form 10 to 40', u'household income form 40 to 75', 
       u'household income 75 and above', u'house value for 20 to 100', u'house value for 100 to 500',
       u'house value 500 or more', u'rent bewteen 300 and 1000', u'rent bewteen 1000 and 2000', u'rent 2000 or more']                     

In [9]:
#This is the list of selected feature types for the second step
age_residents=[u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over']

race_residents=[u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race']

typeof_household_residents=[u'family households', u'nonfamily households']
education_residents= [u'population education high school', u'population education bachelors', u'population education masters', 
                      u'population education phd']
ownorrent_residents = ['owner  occupied units',  u'renter occupied units']
transportationtype_residents = [u'transportation car', u'transportation public', 'tranportation motorcycle',
                              'Transportation Other means', u'number of cars']
income_residents = [u'household income form 10 to 40', u'household income form 40 to 75', u'household income 75 and above']
housing_values_residents = [u'house value for 20 to 100', u'house value for 100 to 500',
       u'house value 500 or more', u'rent bewteen 300 and 1000',
       u'rent bewteen 1000 and 2000', u'rent 2000 or more']        

Lets Consider different groups of population $g=1,2,…,n$ (based on our demographic indicators) and let:


$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$ 

while $Pc(a,g)$ the number of commuters.
 
Let the unknown (subject to fit) complaining behavior be defined by the average number of complains of type $t$ per resident of group $g$ within his/her place of residency be $rc(g,t)$

Let also, $wc(g,t)$ be the number of complains of type $t$ per commuter of type $g$.

Then the total observed number of complains of type $t$ in the area $a$ is:

$$C(a,t)=\sum_{g,t} Pr(a,g) \ rc(g,t) + \sum_{g,t} Pc(a,g) \ wc(g,t) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \text{(1)}$$ 

Then we know $Pr(a,g)$ and $Pc(a,g)$ (those are our regressors), we know the output variable $C(a,t)$ from 311 statistics. We need to fit the $rc(g,t)$, $wc(g,t)$ - slope coefficients of the multivariate linear regression.

This will give us complaining behavior per people of different groups and it will be distinguished by the complaining mode - while at home and while on the way.



###we will procced as follows:

STEP 1) Lasso regression:

Regressors:  

$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$.
            
$Pc(a,g)$  number of commuters in the location a of group $g$

Target variable to be fit: $rc(g,t)$ -   average number of complains of type $t$ per resident of group $g$                                                      within his/her place of residency 

STEP 2) predict the number of complains per capita $wc(g,t)$ from the results of step 1, using equation $(1)$

Using the predicted value $rc(g,t)$ in each area, we are able to get a $wc(g,t)$ prediction (from the formula of the observed total calls by type $C(a,t)$ variable) 

In [10]:
#First set of regressions using all the demographic features for residents and calls by type
#we are defining an innitial threadshold on the total number of calls per type of call
#we request that the normalized number of calls is greater than the mean for all types.
results={}
for typeof in types_of_calls:
    A2=np.append(np.append(resident_features,typeof),'Neighborhood')   #selection of columns
    myframe1=callsbytype_attributes_residents[A2].dropna() 
    if myframe1[typeof].sum() > calls_bytype_normalized.total_calls.mean() :
        results[typeof]={}
        X=myframe1[resident_features]
        Y=myframe1[typeof]
        #LASSO
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        R2_store=[]
        for i in range(-40,40):
            Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
            Lasso.fit(X_train,Y_train)
            R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
            R2_store.append(R2_OS)
        optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
        results[typeof]['best_alpha']=optim_alpha
        Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
        Lasso.fit(X_train,Y_train)
        R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
        results[typeof]['best_R2']=R2_OS
        results[typeof]['sample']=len(myframe1)
        results[typeof]['normalized calls'] = myframe1[typeof].sum()
    else:
        pass       

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:454: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [11]:
#We create a frame to pass the results of the regressions
results_frame = pd.DataFrame(index=range(0,len(results.keys())), columns=['type','best_alpha','R2','sample_size', 'Normalized calls'])

In [12]:
#And pass the results (stored as a dictionary) into the frame
results_frame['type']=results.keys()
results_frame.head()
for i in results_frame.index:
    typeof=results_frame.loc[i,'type']
    results_frame.iloc[i,1]=results[typeof]['best_alpha']
    results_frame.iloc[i,2]=results[typeof]['best_R2']  
    results_frame.iloc[i,3]=results[typeof]['sample']
    results_frame.iloc[i,4]=results[typeof]['normalized calls']    

In [13]:
#Sorting the frame by the R2 results in a ascending form
results_frame.sort('R2',ascending=False,inplace=True)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [14]:
#Displaying the frame; we will use this list of types of calls for the second step
results_frame 

,type,best_alpha,R2,sample_size,Normalized calls
34,FLOORING/STAIRS,0,0.6173,187,0.442795
28,Noise - Residential,1,0.607514,190,4.23941
10,Noise,0,0.58789,190,1.12104
7,UNSANITARY CONDITION,1,0.556622,188,1.3218
19,PLUMBING,1,0.552888,188,1.33045
36,PAINT/PLASTER,1,0.550318,188,1.32788
6,HEAT/HOT WATER,1,0.521671,190,2.77999
23,Building/Use,1,0.491954,188,0.626404
29,DOOR/WINDOW,0,0.477424,186,0.612973
2,WATER LEAK,1,0.475575,188,0.612883


In [15]:
#We create a new fraame subsetting the results one, selecting only the types of complaints
#with a R2 positive result
top_R2=results_frame[results_frame.R2>=0]
top_R2 = top_R2.reset_index(drop=True)

In [16]:
top_R2

,type,best_alpha,R2,sample_size,Normalized calls
0,FLOORING/STAIRS,0,0.6173,187,0.442795
1,Noise - Residential,1,0.607514,190,4.23941
2,Noise,0,0.58789,190,1.12104
3,UNSANITARY CONDITION,1,0.556622,188,1.3218
4,PLUMBING,1,0.552888,188,1.33045
5,PAINT/PLASTER,1,0.550318,188,1.32788
6,HEAT/HOT WATER,1,0.521671,190,2.77999
7,Building/Use,1,0.491954,188,0.626404
8,DOOR/WINDOW,0,0.477424,186,0.612973
9,WATER LEAK,1,0.475575,188,0.612883


In [17]:
#We then proceed to re run lasso, only with the selected types of requests
#using only a selected group of features, in this case age
#and creating a frame to store the resulting cofficients by feature
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(age_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[age_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(age_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=age_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_age = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_age

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,Population under 18,0.000000e+00,-8.930901e-08,0.000000e+00,-5.278336e-08,3.238429e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.982326e-07,7.075701e-07,0.000000e+00,-1.050657e-07,0.000000e+00,-1.389717e-09,-4.717656e-07,6.615595e-07,2.499854e-07,-0.000001
1,population 65 and over,-9.945644e-07,0.000000e+00,0.000000e+00,-1.141830e-07,-3.523755e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-7.754770e-07,-7.130643e-07,0.000000e+00,-3.413896e-07,0.000000e+00,0.000000e+00,0.000000e+00,-4.312124e-07,-2.029068e-07,-0.000014
2,population between 18 and 34,-4.593221e-07,0.000000e+00,-7.017873e-08,3.549444e-08,2.485794e-07,-9.156741e-08,-3.562998e-08,-5.689704e-08,-2.176532e-08,...,1.705973e-07,1.889629e-07,1.257509e-07,-1.876113e-07,-1.196145e-07,-7.055597e-08,2.191100e-07,2.459253e-07,0.000000e+00,0.000002
3,population between 35 to 64,7.606350e-07,0.000000e+00,7.110786e-08,5.020019e-08,-1.416407e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-2.042394e-07,-2.352516e-07,-1.014196e-07,2.477537e-07,0.000000e+00,0.000000e+00,-3.823926e-08,-3.342294e-07,-3.457859e-08,0.000001


In [18]:
#We repeat the same procedure with race features
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(race_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[race_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(race_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=race_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_race = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_race

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,Population asian,2.269507e-07,0.000000e+00,7.909195e-08,2.684366e-08,-1.015518e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.814271e-07,-1.726773e-07,-6.729487e-08,-7.034656e-09,0.000000e+00,0.000000e+00,-2.937698e-08,-1.615191e-07,-6.250053e-08,-2.851088e-06
1,Population white,-9.354442e-08,1.234897e-07,-5.503425e-08,-3.551135e-09,-2.955096e-08,0.000000e+00,6.451540e-09,0.000000e+00,-1.927393e-08,...,-9.923579e-08,-9.695571e-08,0.000000e+00,-1.557958e-08,0.000000e+00,0.000000e+00,0.000000e+00,-7.726760e-08,-4.186788e-08,-1.239572e-06
2,population black,4.499836e-09,-4.598506e-08,-1.944537e-08,-5.857022e-09,4.845360e-08,-5.985186e-09,-1.398974e-09,0.000000e+00,1.983291e-08,...,6.084073e-08,1.016157e-07,0.000000e+00,4.911372e-09,0.000000e+00,-1.541420e-08,-8.396319e-08,7.682067e-08,4.142230e-08,-5.675568e-07
3,population hispanic,6.197665e-08,-7.874524e-08,-2.781380e-08,3.835016e-09,1.160753e-07,-8.147403e-08,-4.127882e-08,-4.263400e-08,-8.694848e-09,...,2.023014e-07,1.750605e-07,2.861835e-08,-5.458773e-08,-9.621329e-08,-4.689902e-08,-1.306119e-07,1.708009e-07,7.709989e-08,1.420569e-08
4,population other race,2.906815e-07,0.000000e+00,7.798474e-07,7.072451e-08,1.171117e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,9.249123e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.360614e-06


In [19]:
#We repeat the same procedure with type of household features
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(typeof_household_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[typeof_household_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(typeof_household_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=typeof_household_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_typeof_household = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_typeof_household

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,family households,5.196505e-07,0.000000e+00,8.260876e-08,-1.334774e-08,1.073116e-07,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,9.487774e-09,9.125947e-09,-3.835316e-08,3.284026e-09,0.000000e+00,-2.973226e-08,-5.790965e-07,1.507152e-08,0.0,-0.000005
1,nonfamily households,-4.990497e-07,2.958718e-07,-1.402846e-07,5.741159e-08,3.675429e-08,-3.764344e-08,0.0,-4.485632e-08,-5.573499e-08,...,0.000000e+00,0.000000e+00,2.896802e-08,-9.181384e-08,-6.085801e-08,-2.546977e-08,4.124144e-07,0.000000e+00,0.0,0.000001


In [20]:
#We repeat the same procedure with education levels  
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(education_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[education_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(education_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=education_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_education = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_education

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,population education bachelors,0.000000e+00,2.947763e-07,-4.002136e-08,1.822192e-07,-1.399126e-07,0.0,0.0,0.0,-4.200915e-08,...,-3.085825e-07,-2.997973e-07,0.000000e+00,-4.094310e-08,0.0,0.000000e+00,1.753912e-07,-2.647742e-07,-1.015426e-07,0.000004
1,population education high school,6.178494e-07,-1.265499e-07,1.332577e-07,-3.184174e-08,2.367085e-07,0.0,0.0,0.0,1.686023e-08,...,2.454450e-08,3.830178e-08,-3.556215e-08,-9.749594e-10,0.0,-1.499675e-08,-5.031984e-07,3.923098e-08,0.000000e+00,-0.000005
2,population education masters,-3.805191e-07,0.000000e+00,0.000000e+00,-2.595743e-07,-3.714614e-07,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,2.069222e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000011
3,population education phd,0.000000e+00,0.000000e+00,0.000000e+00,4.403008e-08,2.112875e-06,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,-1.181570e-06,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [21]:
#We repeat the same procedure with type of resident (owner vs renter) 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(ownorrent_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[ownorrent_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(ownorrent_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=ownorrent_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_ownorrent = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_ownorrent

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,owner occupied units,1.544335e-07,0.000000e+00,1.183825e-07,-3.840142e-08,-4.606417e-07,6.522705e-08,7.291733e-08,0.000000e+00,0.000000e+00,...,-9.940485e-07,-9.136723e-07,-1.311642e-07,9.670246e-08,0.000000e+00,9.487905e-09,-3.264000e-08,-8.486262e-07,-3.653909e-07,-7.981502e-06
1,renter occupied units,-8.116522e-08,2.971257e-08,-8.750511e-08,4.845951e-08,1.996565e-07,-1.367065e-07,-7.111565e-08,-1.052460e-07,-4.032408e-08,...,2.991655e-07,2.812409e-07,4.641035e-08,-1.027050e-07,-1.904112e-07,-8.035305e-08,0.000000e+00,2.688616e-07,1.176725e-07,6.761064e-07


In [22]:
#We repeat the same procedure with transportation type for residents 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(transportationtype_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[transportationtype_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(transportationtype_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=transportationtype_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_transportationtype = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_transportationtype

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,Transportation Other means,-8.843289e-07,3.150275e-07,0.000000e+00,1.585566e-07,-5.237917e-07,0.000000e+00,0.000000e+00,0.000000e+00,-6.225841e-09,...,-6.382712e-07,-5.195703e-07,0.000000e+00,-4.570725e-08,0.000000e+00,0.000000e+00,5.639161e-07,-4.936580e-07,-2.521185e-08,-4.706301e-06
1,number of cars,0.000000e+00,0.000000e+00,0.000000e+00,-5.480822e-07,1.603819e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,7.803727e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.195972e-07
2,tranportation motorcycle,0.000000e+00,0.000000e+00,0.000000e+00,-1.131487e-05,-3.716544e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,7.731510e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,transportation car,3.166986e-07,0.000000e+00,2.558025e-07,4.151185e-07,-6.231245e-07,9.610404e-08,8.644191e-08,6.335372e-08,8.927997e-09,...,-8.793122e-07,-8.011174e-07,-1.560995e-07,-5.272374e-07,1.027686e-07,1.691207e-08,-3.968668e-07,-7.541658e-07,-2.800789e-07,-9.221666e-06
4,transportation public,2.603340e-07,0.000000e+00,-1.640667e-08,1.084019e-08,2.948554e-07,-7.917151e-08,-4.256714e-08,-6.659555e-08,-1.564363e-08,...,2.550711e-07,2.195035e-07,1.382353e-08,-5.108344e-08,-1.333195e-07,-6.000087e-08,-1.033108e-07,2.165067e-07,4.774937e-08,7.472419e-07


In [23]:
#We repeat the same procedure with income levels 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(income_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[income_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(income_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=income_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_income = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_income

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,household income 75 and above,-5.401390e-07,3.495196e-07,-3.619368e-09,6.695700e-08,-1.836970e-07,0.0,0.0,0.0,-2.785469e-08,...,-5.629686e-07,-5.210452e-07,0.0,-1.262546e-07,0.0,0.000000e+00,1.952299e-07,-4.784784e-07,-2.026457e-07,-0.000001
1,household income form 10 to 40,-7.523954e-07,0.000000e+00,-5.701433e-10,2.112986e-07,8.908542e-07,0.0,0.0,0.0,0.000000e+00,...,9.330098e-07,8.745939e-07,0.0,-5.793388e-07,0.0,-1.120431e-07,-3.571552e-07,8.117085e-07,3.226801e-07,0.000004
2,household income form 40 to 75,2.151623e-06,0.000000e+00,0.000000e+00,-2.687325e-07,-4.902549e-07,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,7.020838e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000011


In [25]:
#Finally we repeat it for housing values  
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(housing_values_residents, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_residents[A2].dropna() 
    X=myframe1[housing_values_residents]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(housing_values_residents)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=housing_values_residents
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_housing_values = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_housing_values

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK,total_calls
0,house value 500 or more,0.000000e+00,0.000000e+00,0.000000e+00,-9.342912e-08,-3.369642e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-9.399455e-07,-8.915244e-07,-6.400767e-09,5.906147e-08,0.000000e+00,0.000000e+00,0.000000e+00,-8.135118e-07,-3.120906e-07,-0.000006
1,house value for 100 to 500,1.158093e-07,0.000000e+00,1.444345e-07,-1.798678e-08,-4.651799e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-7.635463e-07,-5.374152e-07,-8.894885e-08,1.794756e-07,0.000000e+00,0.000000e+00,-8.783264e-08,-5.349025e-07,-1.620469e-07,-0.000010
2,house value for 20 to 100,0.000000e+00,0.000000e+00,0.000000e+00,-1.589844e-08,4.524868e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,-1.252616e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000024
3,rent 2000 or more,-5.992180e-07,6.510951e-07,-6.851876e-08,1.483605e-07,-1.416826e-07,0.000000e+00,0.000000e+00,0.000000e+00,-4.751063e-08,...,-4.180953e-08,0.000000e+00,0.000000e+00,-9.488525e-08,0.000000e+00,0.000000e+00,6.615929e-07,0.000000e+00,0.000000e+00,0.000001
4,rent bewteen 1000 and 2000,6.634657e-07,0.000000e+00,0.000000e+00,2.244769e-08,2.772824e-07,-3.873686e-09,-2.569397e-09,-1.343204e-08,0.000000e+00,...,4.185607e-07,2.872724e-07,0.000000e+00,-4.297007e-08,-8.940854e-08,-2.869311e-08,-3.124633e-07,3.094584e-07,1.256490e-07,0.000001
5,rent bewteen 300 and 1000,-7.774120e-07,0.000000e+00,-1.625348e-07,2.636541e-08,2.381383e-07,-1.030821e-07,-2.701257e-08,0.000000e+00,0.000000e+00,...,3.440705e-07,4.846341e-07,4.765830e-08,-1.396880e-07,0.000000e+00,-1.070483e-07,0.000000e+00,4.070312e-07,1.780477e-07,-0.000001


In [28]:
#we concatenate all created frames into a single one to export the info to a CSV

frames = [lasso_coefficients_age, lasso_coefficients_education, lasso_coefficients_housing_values, lasso_coefficients_income, 
          lasso_coefficients_ownorrent, lasso_coefficients_race, lasso_coefficients_transportationtype,
          lasso_coefficients_typeof_household]

lasso_coefficients_residents = pd.concat(frames)

In [32]:
lasso_coefficients_residents = lasso_coefficients_residents.reset_index(drop=True)
del(lasso_coefficients_residents['total_calls'])
lasso_coefficients_residents

request type,regressor,Blocked Driveway,Broken Muni Meter,Building/Use,Consumer Complaint,DOOR/WINDOW,Damaged Tree,Dead Tree,Derelict Vehicle,Derelict Vehicles,...,Overgrown Tree/Branches,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Snow,Taxi Complaint,UNSANITARY CONDITION,WATER LEAK
0,Population under 18,0.000000e+00,-8.930901e-08,0.000000e+00,-5.278336e-08,3.238429e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,6.982326e-07,7.075701e-07,0.000000e+00,-1.050657e-07,0.000000e+00,-1.389717e-09,-4.717656e-07,6.615595e-07,2.499854e-07
1,population 65 and over,-9.945644e-07,0.000000e+00,0.000000e+00,-1.141830e-07,-3.523755e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-7.754770e-07,-7.130643e-07,0.000000e+00,-3.413896e-07,0.000000e+00,0.000000e+00,0.000000e+00,-4.312124e-07,-2.029068e-07
2,population between 18 and 34,-4.593221e-07,0.000000e+00,-7.017873e-08,3.549444e-08,2.485794e-07,-9.156741e-08,-3.562998e-08,-5.689704e-08,-2.176532e-08,...,-7.342568e-08,1.705973e-07,1.889629e-07,1.257509e-07,-1.876113e-07,-1.196145e-07,-7.055597e-08,2.191100e-07,2.459253e-07,0.000000e+00
3,population between 35 to 64,7.606350e-07,0.000000e+00,7.110786e-08,5.020019e-08,-1.416407e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-2.042394e-07,-2.352516e-07,-1.014196e-07,2.477537e-07,0.000000e+00,0.000000e+00,-3.823926e-08,-3.342294e-07,-3.457859e-08
4,population education bachelors,0.000000e+00,2.947763e-07,-4.002136e-08,1.822192e-07,-1.399126e-07,0.000000e+00,0.000000e+00,0.000000e+00,-4.200915e-08,...,0.000000e+00,-3.085825e-07,-2.997973e-07,0.000000e+00,-4.094310e-08,0.000000e+00,0.000000e+00,1.753912e-07,-2.647742e-07,-1.015426e-07
5,population education high school,6.178494e-07,-1.265499e-07,1.332577e-07,-3.184174e-08,2.367085e-07,0.000000e+00,0.000000e+00,0.000000e+00,1.686023e-08,...,0.000000e+00,2.454450e-08,3.830178e-08,-3.556215e-08,-9.749594e-10,0.000000e+00,-1.499675e-08,-5.031984e-07,3.923098e-08,0.000000e+00
6,population education masters,-3.805191e-07,0.000000e+00,0.000000e+00,-2.595743e-07,-3.714614e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.069222e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,population education phd,0.000000e+00,0.000000e+00,0.000000e+00,4.403008e-08,2.112875e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.181570e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,house value 500 or more,0.000000e+00,0.000000e+00,0.000000e+00,-9.342912e-08,-3.369642e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-9.399455e-07,-8.915244e-07,-6.400767e-09,5.906147e-08,0.000000e+00,0.000000e+00,0.000000e+00,-8.135118e-07,-3.120906e-07
9,house value for 100 to 500,1.158093e-07,0.000000e+00,1.444345e-07,-1.798678e-08,-4.651799e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.170568e-10,-7.635463e-07,-5.374152e-07,-8.894885e-08,1.794756e-07,0.000000e+00,0.000000e+00,-8.783264e-08,-5.349025e-07,-1.620469e-07


In [33]:
lasso_coefficients_residents.to_csv('lasso_coefficients_residents_by_feature_type.csv')

In [18]:
demographics_NTA_NYC_workers.columns

Index([u'Neighborhood', u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n',
       u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n',
       u'family households_n', u'nonfamily households_n',
       u'population education high school_n',
       u'population education bachelors_n', u'population education masters_n',
       u'population education phd_n', u'household income less than 10_n',
       u'owner  occupied units_n', u'renter occupied units_n',
       u'cars per capita', u'transportation car_n', u'transportation public_n',
       u'tranportation motorcycle_n', u'total workers',
       u'household income form 10 to 40_n',
       u'household income form 40 to 75_n', u'household income 75 and above_n',
       u'house value for 20 to 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent bewteen 300 and 1000_n',
       u'rent bew

In [19]:
#Defining the selected features for the innitial resident lasso regression
#we will use this innitial regression to determine a selected list of types of complaints 
#that meet a certain threadshold to then perform a selective regression by feature types
workers_features = [u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n',
       u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n',
       u'family households_n', u'nonfamily households_n',
       u'population education high school_n',
       u'population education bachelors_n', u'population education masters_n',
       u'population education phd_n', u'household income less than 10_n',
       u'owner  occupied units_n', u'renter occupied units_n',
       u'cars per capita', u'transportation car_n', u'transportation public_n',
       u'tranportation motorcycle_n', u'total workers',
       u'household income form 10 to 40_n',
       u'household income form 40 to 75_n', u'household income 75 and above_n',
       u'house value for 20 to 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent bewteen 300 and 1000_n',
       u'rent bewteen 1000 and 2000_n', u'Transportation Other means_n']       

In [20]:
#This is the list of selected feature types for the second step
age_workers=[u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n']

race_workers=[u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n']

typeof_household_workers=[u'family households_n', u'nonfamily households_n']
education_workers= [u'population education high school_n', u'population education bachelors_n', u'population education masters_n', 
                      u'population education phd_n']
ownorrent_workers = ['owner  occupied units_n',  u'renter occupied units_n']
transportationtype_workers = [u'transportation car_n', u'transportation public_n', 'tranportation motorcycle_n',
                              'Transportation Other means_n']
income_workers = [u'household income form 10 to 40_n', u'household income form 40 to 75_n', u'household income 75 and above_n']
housing_values_workers = [u'house value for 20 to 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent bewteen 300 and 1000_n',
       u'rent bewteen 1000 and 2000_n', u'rent 2000 or more_n']        

In [52]:
#First set of regressions using all the demographic features for residents and calls by type
#we are defining an innitial threadshold on the total number of calls per type of call
#we request that the normalized number of calls is greater than the mean for all types.
results={}
for typeof in types_of_calls_workers:
    A2=np.append(np.append(workers_features,typeof),'Neighborhood')   #selection of columns
    myframe1=callsbytype_attributes_workers[A2].dropna() 
    if myframe1[typeof].sum() > calls_bytype_normalized_workers.total_calls.mean()*0.25 :
        results[typeof]={}
        X=myframe1[workers_features]
        Y=myframe1[typeof]
        #LASSO
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        R2_store=[]
        for i in range(-40,40):
            Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
            Lasso.fit(X_train,Y_train)
            R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
            R2_store.append(R2_OS)
        optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
        results[typeof]['best_alpha']=optim_alpha
        Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
        Lasso.fit(X_train,Y_train)
        R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
        results[typeof]['best_R2']=R2_OS
        results[typeof]['sample']=len(myframe1)
        results[typeof]['normalized calls'] = myframe1[typeof].sum()
    else:
        pass       

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [53]:
#We create a frame to pass the results of the regressions
results_frame = pd.DataFrame(index=range(0,len(results.keys())), columns=['type','best_alpha','R2','sample_size', 'Normalized calls'])

In [54]:
#And pass the results (stored as a dictionary) into the frame
results_frame['type']=results.keys()
results_frame.head()
for i in results_frame.index:
    typeof=results_frame.loc[i,'type']
    results_frame.iloc[i,1]=results[typeof]['best_alpha']
    results_frame.iloc[i,2]=results[typeof]['best_R2']  
    results_frame.iloc[i,3]=results[typeof]['sample']
    results_frame.iloc[i,4]=results[typeof]['normalized calls']    

In [55]:
#Sorting the frame by the R2 results in a ascending form
results_frame.sort('R2',ascending=False,inplace=True)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [56]:
results_frame

,type,best_alpha,R2,sample_size,Normalized calls
45,Root/Sewer/Sidewalk Condition,2,0.142014,183,0.999572
7,Elevator,0,0.109328,167,0.672961
10,NONCONST,0,0.106331,186,0.7849
49,GENERAL CONSTRUCTION,1,0.0994561,185,1.43391
17,Noise,5,0.092885,190,2.29581
46,PAINT - PLASTER,1,0.0846903,181,1.0872
3,Taxi Complaint,1,0.0842711,184,0.443597
30,Missed Collection (All Materials),2,0.0786004,187,2.17723
34,Traffic,1,0.0609694,189,0.215175
62,Plumbing,11,0.0445182,186,0.296741


In [57]:
#We create a new fraame subsetting the results one, selecting only the types of complaints
#with a R2 positive result
top_R2=results_frame[results_frame.R2>=0]
top_R2 = top_R2.reset_index(drop=True)

In [58]:
top_R2

,type,best_alpha,R2,sample_size,Normalized calls
0,Root/Sewer/Sidewalk Condition,2,0.142014,183,0.999572
1,Elevator,0,0.109328,167,0.672961
2,NONCONST,0,0.106331,186,0.7849
3,GENERAL CONSTRUCTION,1,0.0994561,185,1.43391
4,Noise,5,0.092885,190,2.29581
5,PAINT - PLASTER,1,0.0846903,181,1.0872
6,Taxi Complaint,1,0.0842711,184,0.443597
7,Missed Collection (All Materials),2,0.0786004,187,2.17723
8,Traffic,1,0.0609694,189,0.215175
9,Plumbing,11,0.0445182,186,0.296741


In [59]:
#We then proceed to re run lasso, only with the selected types of requests
#using only a selected group of features, in this case age
#and creating a frame to store the resulting cofficients by feature
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(age_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[age_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(age_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=age_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_age = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_age

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,population 65 and over_n,-2.105878e-07,0.000000e+00,-1.177654e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-2.846493e-07,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,population between 18 and 34_n,6.686349e-07,0.000000e+00,2.463174e-07,0.000000e+00,0.000000e+00,0.0,-1.967188e-07,4.097123e-07,0.000000e+00,...,0.000000e+00,0.0,-9.743842e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,population between 35 to 64_n,-4.417389e-07,-8.698442e-08,-1.476516e-07,-3.466838e-08,-4.057175e-07,0.0,0.000000e+00,-2.066101e-07,-3.651094e-08,...,-3.079289e-08,0.0,0.000000e+00,-1.246620e-08,-7.487130e-09,-1.634543e-10,-1.395395e-08,-8.972333e-10,-3.394927e-09,-6.966658e-08


In [60]:
#We repeat the same procedure with race features
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(race_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[race_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(race_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=race_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_race = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_race

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,Population asian_n,-2.203966e-06,0.000000e+00,-6.009498e-07,-5.757529e-07,0.000000e+00,0.0,0.000000e+00,-6.831885e-07,0.000000e+00,...,-3.323897e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,Population white_n,0.000000e+00,0.000000e+00,4.075546e-08,0.000000e+00,-4.403258e-07,0.0,6.922927e-07,6.773906e-08,0.000000e+00,...,0.000000e+00,0.0,5.664202e-08,-1.295648e-08,0.000000e+00,-6.210399e-10,2.221566e-09,-8.690779e-10,0.000000e+00,2.646031e-07
2,population black_n,-1.690852e-07,0.000000e+00,-2.735904e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-7.779219e-08,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.802509e-07
3,population hispanic_n,1.154336e-06,-1.968742e-07,4.056429e-07,2.251987e-07,0.000000e+00,0.0,-1.592938e-06,2.245933e-07,-7.540225e-08,...,1.062541e-07,0.0,-2.364725e-07,0.000000e+00,-1.451859e-08,0.000000e+00,-3.408076e-08,0.000000e+00,-6.821615e-09,-8.606522e-07
4,population other _n,0.000000e+00,0.000000e+00,-1.008275e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.681200e-07,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [61]:
#We repeat the same procedure with type of household features
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(typeof_household_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[typeof_household_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(typeof_household_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=typeof_household_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)

lasso_coefficients_typeof_household = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_typeof_household

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,family households_n,-4.297277e-07,-7.496123e-08,-1.540250e-07,-3.109200e-07,-3.584063e-07,0.0,-1.138199e-07,-1.050420e-07,-3.409485e-08,...,-1.349979e-07,-1.902039e-09,-5.381269e-08,-1.107922e-08,-6.680315e-09,-7.405970e-10,-1.173512e-08,-5.485326e-08,-2.982110e-09,-5.723855e-08
1,nonfamily households_n,5.565332e-07,0.000000e+00,2.030519e-07,3.960494e-07,0.000000e+00,0.0,0.000000e+00,1.327488e-07,0.000000e+00,...,1.560901e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.616986e-08,0.000000e+00,0.000000e+00


In [62]:
#We repeat the same procedure with education levels  
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(education_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[education_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(education_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=education_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_education = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_education

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,population education bachelors_n,3.485274e-07,0.000000e+00,1.482420e-07,2.337051e-07,0.000000e+00,0.0,0.000000e+00,1.799801e-07,0.000000e+00,...,5.886494e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,population education high school_n,-5.382026e-07,-1.987237e-07,-2.943536e-07,-3.764686e-07,-9.337628e-07,0.0,-3.384917e-07,-2.213434e-07,-7.184897e-08,...,-1.510466e-07,0.0,-1.514517e-07,-3.060483e-08,-1.335994e-08,0.0,-2.931754e-08,-2.402829e-09,-8.403657e-09,-1.689732e-07
2,population education masters_n,0.000000e+00,0.000000e+00,3.280963e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,2.154498e-07,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,population education phd_n,0.000000e+00,0.000000e+00,-1.346108e-06,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-1.817819e-06,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [63]:
#We repeat the same procedure with type of resident (owner vs renter) 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(ownorrent_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[ownorrent_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(ownorrent_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=ownorrent_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_ownorrent = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_ownorrent

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,owner occupied units_n,-1.392931e-06,0.000000e+00,-2.448694e-07,-4.808900e-07,-6.171690e-07,0.0,6.787673e-07,-2.701262e-07,0.000000e+00,...,-3.060106e-07,0.000000e+00,1.816294e-07,-1.785740e-08,0.000000e+00,0.000000e+00,0.000000e+00,-8.692112e-08,0.000000e+00,1.048215e-07
1,renter occupied units_n,8.622764e-07,-8.521157e-08,1.451396e-07,2.781266e-07,0.000000e+00,0.0,-5.689760e-07,1.615940e-07,-3.249894e-08,...,1.700822e-07,-8.349759e-10,-1.819626e-07,0.000000e+00,-7.064264e-09,-1.561030e-09,-1.389684e-08,5.478995e-08,-3.123452e-09,-1.329074e-07


In [64]:
#We repeat the same procedure with transportation type for workers 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(transportationtype_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[transportationtype_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(transportationtype_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=transportationtype_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_transportationtype = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_transportationtype

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,Transportation Other means_n,0.000000,0.000000e+00,7.451220e-07,3.710572e-07,0.000000e+00,0.0,0.000000e+00,4.332321e-07,0.000000e+00,...,1.259425e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,tranportation motorcycle_n,0.000000,0.000000e+00,-1.824180e-05,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-7.589389e-06,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,transportation car_n,-0.000001,0.000000e+00,-3.084059e-07,-1.935686e-07,-7.471071e-07,0.0,1.664429e-08,-3.152866e-07,-6.076010e-08,...,-1.002319e-07,0.0,0.000000e+00,-2.186301e-08,-1.096991e-08,0.0,0.000000e+00,-2.162033e-09,-5.822575e-09,-1.177096e-07
3,transportation public_n,0.000002,-1.776439e-07,3.420910e-07,0.000000e+00,0.000000e+00,0.0,-3.348741e-07,3.068431e-07,0.000000e+00,...,0.000000e+00,0.0,-1.544684e-07,0.000000e+00,0.000000e+00,0.0,-2.869636e-08,0.000000e+00,0.000000e+00,-1.307430e-08


In [65]:
#We repeat the same procedure with income levels 
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(income_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[income_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(income_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=income_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_income = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_income

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,household income 75 and above_n,1.495374e-07,1.003988e-07,8.379597e-08,1.225813e-07,-2.178981e-07,-7.371054e-10,2.783753e-07,5.868997e-08,0.000000e+00,...,5.610709e-08,-3.589706e-09,8.538728e-08,0.000000e+00,0.000000e+00,-2.147222e-09,1.630477e-09,0.000000e+00,0.000000e+00,1.167189e-07
1,household income form 10 to 40_n,1.318080e-07,-2.782669e-07,-9.412593e-09,0.000000e+00,0.000000e+00,0.000000e+00,-6.423897e-07,8.341323e-08,-3.618250e-08,...,0.000000e+00,0.000000e+00,-2.068752e-07,-1.233498e-08,-7.768360e-09,0.000000e+00,-1.780785e-08,-5.733825e-10,-3.469872e-09,-2.714010e-07
2,household income form 40 to 75_n,-5.438087e-07,0.000000e+00,-1.922662e-07,-3.167482e-07,0.000000e+00,0.000000e+00,0.000000e+00,-2.467916e-07,0.000000e+00,...,-1.610824e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [66]:
#Finally we repeat it for housing values  
lasso_coefficients =  pd.DataFrame(columns=['regressor','coefficient', 'request type'])
for i in range (0, len(top_R2)):
    A2=np.append(np.append(housing_values_workers, top_R2['type'][i]),'Neighborhood')  
    myframe1 = callsbytype_attributes_workers[A2].dropna() 
    X=myframe1[housing_values_workers]
    Y=myframe1[top_R2['type'][i]]
    #LASSO
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=top_R2['best_alpha'][i])
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
    lc = pd.DataFrame(index=range(0,len(housing_values_workers)), columns=['regressor','coefficient', 'request type'])
    lc['regressor']=housing_values_workers
    lc['coefficient']=Lasso.coef_
    lc['request type']=top_R2['type'][i]
    lasso_coefficients = pd.concat([lasso_coefficients, lc]).reset_index(drop=True)
lasso_coefficients_housing_values = pd.pivot_table(lasso_coefficients, values='coefficient', index=['regressor'], columns=['request type']).reset_index()
lasso_coefficients_housing_values

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,house value 500 or more_n,-2.858438e-07,0.000000e+00,-2.645755e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-1.364793e-07,0.000000e+00,...,-6.011476e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-7.089486e-10,0.000000e+00,1.606494e-09,0.000000e+00,1.974963e-07
1,house value for 100 to 500_n,-2.334198e-07,0.000000e+00,-3.052299e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-1.779998e-07,-4.797207e-08,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-8.835053e-09,0.000000e+00,0.000000e+00,-8.517613e-09,0.000000e+00,-1.657758e-08
2,house value for 20 to 100_n,4.120915e-06,0.000000e+00,2.058214e-06,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.188456e-06,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,rent 2000 or more_n,5.801637e-07,0.000000e+00,4.169797e-07,4.338490e-07,0.000000e+00,0.0,1.952902e-07,2.522065e-07,0.000000e+00,...,2.333133e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.049891e-08,0.000000e+00,0.000000e+00
4,rent bewteen 1000 and 2000_n,-1.045190e-06,-1.709780e-08,-1.394053e-07,-3.654063e-07,-4.136124e-07,0.0,0.000000e+00,-1.830360e-07,0.000000e+00,...,-1.879326e-07,0.0,-6.239914e-08,-1.268557e-08,0.000000e+00,0.000000e+00,-1.342145e-08,0.000000e+00,-3.411399e-09,0.000000e+00
5,rent bewteen 300 and 1000_n,1.442253e-06,-1.297576e-07,4.202772e-07,2.606698e-07,0.000000e+00,0.0,-4.034024e-07,3.930503e-07,0.000000e+00,...,2.165121e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.265755e-07


In [67]:
#we concatenate all created frames into a single one to export the info to a CSV

frames = [lasso_coefficients_age, lasso_coefficients_education, lasso_coefficients_housing_values, lasso_coefficients_income, 
          lasso_coefficients_ownorrent, lasso_coefficients_race, lasso_coefficients_transportationtype,
          lasso_coefficients_typeof_household]

lasso_coefficients_workers = pd.concat(frames)

In [68]:
lasso_coefficients_workers = lasso_coefficients_workers.reset_index(drop=True)
lasso_coefficients_workers

request type,regressor,DOOR/WINDOW,Derelict Vehicle,Elevator,GENERAL CONSTRUCTION,HEATING,Indoor Air Quality,Missed Collection (All Materials),NONCONST,Noise,...,PAINT - PLASTER,Plumbing,Root/Sewer/Sidewalk Condition,SAFETY,Sidewalk Condition,Special Projects Inspection Team (SPIT),Standing Water,Taxi Complaint,Traffic,Traffic Signal Condition
0,population 65 and over_n,-2.105878e-07,0.000000e+00,-1.177654e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.846493e-07,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,population between 18 and 34_n,6.686349e-07,0.000000e+00,2.463174e-07,0.000000e+00,0.000000e+00,0.000000e+00,-1.967188e-07,4.097123e-07,0.000000e+00,...,0.000000e+00,0.000000e+00,-9.743842e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,population between 35 to 64_n,-4.417389e-07,-8.698442e-08,-1.476516e-07,-3.466838e-08,-4.057175e-07,0.000000e+00,0.000000e+00,-2.066101e-07,-3.651094e-08,...,-3.079289e-08,0.000000e+00,0.000000e+00,-1.246620e-08,-7.487130e-09,-1.634543e-10,-1.395395e-08,-8.972333e-10,-3.394927e-09,-6.966658e-08
3,population education bachelors_n,3.485274e-07,0.000000e+00,1.482420e-07,2.337051e-07,0.000000e+00,0.000000e+00,0.000000e+00,1.799801e-07,0.000000e+00,...,5.886494e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,population education high school_n,-5.382026e-07,-1.987237e-07,-2.943536e-07,-3.764686e-07,-9.337628e-07,0.000000e+00,-3.384917e-07,-2.213434e-07,-7.184897e-08,...,-1.510466e-07,0.000000e+00,-1.514517e-07,-3.060483e-08,-1.335994e-08,0.000000e+00,-2.931754e-08,-2.402829e-09,-8.403657e-09,-1.689732e-07
5,population education masters_n,0.000000e+00,0.000000e+00,3.280963e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.154498e-07,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,population education phd_n,0.000000e+00,0.000000e+00,-1.346108e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.817819e-06,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,house value 500 or more_n,-2.858438e-07,0.000000e+00,-2.645755e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.364793e-07,0.000000e+00,...,-6.011476e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.089486e-10,0.000000e+00,1.606494e-09,0.000000e+00,1.974963e-07
8,house value for 100 to 500_n,-2.334198e-07,0.000000e+00,-3.052299e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.779998e-07,-4.797207e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.835053e-09,0.000000e+00,0.000000e+00,-8.517613e-09,0.000000e+00,-1.657758e-08
9,house value for 20 to 100_n,4.120915e-06,0.000000e+00,2.058214e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.188456e-06,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [69]:
lasso_coefficients_workers.to_csv('lasso_coefficients_workers_by_feature_type.csv')